In [6]:
using Pipe
push!(LOAD_PATH, "../word-embeddings2")
using WordEmbeddings
we = @pipe load_embeddings("../word-embeddings2/word_emb_data/embeddings-scaled.EMBEDDING_SIZE=50.txt") |> WE(_...);


In [41]:
words = """
France
Germany
England
Britan

China
Australia
Japan

Peru
Brasil
USA
Canada
UK

running
jumping
hiking
fishing
sailing
diving
cycling
swimming
golfing
fighting
reading
working
rowing

""" |> split;


In [42]:
words = convert(Vector{String}, words)
xs = eval_word_embeddings(we, words)';

In [28]:
function show_posterior_prob(p_ij,xs, words)
    order = @pipe findmax(p_ij,2)[2] |> map(ind->ind2sub(size(p_ij),ind)[2],_)
    word_indexes = sort([1:length(order);], by=ii->order[ii])
    for ii in word_indexes
        word = rpad(words[ii],8," ")
        ps = @pipe p_ij[ii,:] |> round(_,2) #|> string |> rpad(_,4,"0")

        print("$word\t|  ")
        for p in ps 
            print(" $p ")
        end
        println()
    end
end

show_posterior_prob (generic function with 1 method)

In [71]:
using GaussianMixtures
gmm=GMM(4, size(xs,2), kind=:diag)
gmm.μ = xs[rand(1:size(xs,1),gmm.n),:]

#gmm.μ = [xs[1,:]; xs[end,:];]

lLs = em!(gmm, xs; nIter=20)
#println(lLs)
e^lLs[end]

1.8993562176414902

In [27]:
p_ij, ll = gmmposterior(gmm, xs)
show_posterior_prob(p_ij,xs, words)

LoadError: gmmposterior not defined
while loading In[27], in expression starting on line 1

In [1]:

using PyCall

In [2]:
@pyimport sklearn
@pyimport sklearn.mixture as sklm


In [51]:
gmm=sklm.GMM(n_components=3, covariance_type="full", n_init=15) 
gmm[:fit](xs)

PyObject GMM(covariance_type='full', init_params='wmc', min_covar=0.001,
  n_components=3, n_init=15, n_iter=100, params='wmc', random_state=None,
  thresh=0.01)

In [52]:
p_ij = gmm[:predict_proba](xs)
show_posterior_prob(p_ij, xs, words)

working 	|   1.0  0.0  0.0 
reading 	|   1.0  0.0  0.0 
fighting	|   1.0  0.0  0.0 
swimming	|   1.0  0.0  0.0 
diving  	|   1.0  0.0  0.0 
sailing 	|   1.0  0.0  0.0 
fishing 	|   1.0  0.0  0.0 
jumping 	|   1.0  0.0  0.0 
running 	|   1.0  0.0  0.0 
UK      	|   0.0  1.0  0.0 
France  	|   0.0  1.0  0.0 
Canada  	|   0.0  1.0  0.0 
Peru    	|   0.0  1.0  0.0 
Japan   	|   0.0  1.0  0.0 
Australia	|   0.0  1.0  0.0 
China   	|   0.0  1.0  0.0 
England 	|   0.0  1.0  0.0 
Germany 	|   0.0  1.0  0.0 
USA     	|   0.0  0.0  1.0 
Brasil  	|   0.0  0.0  1.0 
hiking  	|   0.0  0.0  1.0 
cycling 	|   0.0  0.0  1.0 
golfing 	|   0.0  0.0  1.0 
Britan  	|   0.0  0.0  1.0 
rowing  	|   0.0  0.0  1.0 
